In [ ]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 12.1 MB/s eta 0:00:00


In [ ]:
!pip install bayesian-optimization

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# data handling
import numpy as np
import pandas as pd

# scoring
import sklearn.metrics as metrics
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

# use model
# from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

# bayesian optimization
# from bayes_opt import BayesianOptimization

# make events
from itertools import product

# ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from sklearn.ensemble import VotingClassifier

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [ ]:
# 데이터 불러오기
train = pd.read_csv("C:/bad_web/data/train_dataset.csv")
test = pd.read_csv("C:/bad_web/data/test_dataset_v01.csv")
submission = pd.read_csv("C:/bad_web/data/sample_submission.csv")

# 필요없는 열 제거
train.drop(columns = ['Unnamed: 0'], inplace = True)
test.drop(columns = ['Unnamed: 0'], inplace = True)

# 악성이면 1, 정상이면 1로 바꾸기
train['Result_v1'] = train['Result_v1'].apply(lambda x: -1 if x == 'malicious' else 1)

In [ ]:
# 학습데이터의 양을 임의로 늘려보기
train1 = pd.read_csv('train.csv')
train2 = pd.read_csv('train.csv')
train3 = pd.read_csv('train.csv')
train_ = pd.concat([train1,train2])
train = pd.concat([train_, train3])
test = pd.read_csv('test.csv')

In [ ]:
train.shape

(10992, 24)

In [ ]:
train['label'] = train['label'].apply(lambda x : 0 if x == 'malicious' else 1)

In [ ]:
test.head()

,id,url_len,url_num_hyphens_dom,url_path_len,url_domain_len,url_hostname_len,url_num_dots,url_num_underscores,url_query_len,url_num_query_para,...,html_num_tags('iframe'),html_num_tags('script'),html_num_tags('embed'),html_num_tags('object'),html_num_tags('div'),html_num_tags('head'),html_num_tags('body'),html_num_tags('form'),html_num_tags('a'),html_num_tags('applet')
0,0,97,0,80.0,17.0,17.0,4,0,0,0,...,0,5.0,0,0,12,1,1,1.0,3.0,0
1,1,37,0,22.0,15.0,15.0,3,1,0,0,...,1,NaN,0,0,158,1,1,NaN,NaN,0
2,2,44,0,NaN,20.0,20.0,3,3,0,0,...,0,6.0,3,3,34,1,1,0.0,136.0,0
3,3,40,0,14.0,26.0,26.0,2,0,0,0,...,0,NaN,0,0,29,1,1,1.0,2.0,0
4,4,43,2,NaN,26.0,26.0,2,0,0,0,...,0,NaN,0,0,41,1,1,NaN,NaN,0


In [ ]:
def select_dataset(data,config) :

    data.reset_index(drop = True, inplace = True)

    one_columns = ['url_chinese_present',"html_num_tags('applet')"]
    one_distribute = ['url_query_len','url_num_query_para']
    all_columns = list(data.columns)
    
    if config['one_column_on'] == False :
        for i in one_columns :
            all_columns.remove(i)
        
    if config['url_len_off'] == False :
        all_columns.remove('url_len')
    
    if config['url_entropy_off'] == False :
        all_columns.remove('url_entropy')
    
    if config['url_port_off'] == False :
        all_columns.remove('url_port')
    
    if config['iframe_off'] == False :
        all_columns.remove("html_num_tags('iframe')")
    
    if config['head_off'] == False :
        all_columns.remove("html_num_tags('head')")
    
    if config['body_off'] == False :
        all_columns.remove("html_num_tags('body')")
        
    
    if config['one_distribute_on'] == False :
        for i in one_distribute :
            all_columns.remove(i)

    data = data[all_columns]

    if config['drop_duplicate'] == True :
        data.drop_duplicates(inplace = True)
        data.reset_index(drop = True, inplace = True)
    
    return data

In [ ]:
#testdata결측치 채우기
X = test
imp = IterativeImputer(estimator = RandomForestRegressor(verbose=0, random_state = 42),
                        max_iter=10,
                        verbose = 0,
                        imputation_order= 'ascending',
                        random_state=0)
X_imp=pd.DataFrame(imp.fit_transform(X))
X_imp.columns = X.columns
test = X_imp

### 1) catboost 모델 이용

- 점수 1등의 경우의 수
- (False, True, True, False, True, True, True, True, True)	0.960000

In [ ]:

 config_cb = dict()
config_cb['one_column_on'] =False 
config_cb['url_len_off'] =True
config_cb['url_entropy_off'] =True
config_cb['url_port_off'] =False 
config_cb['iframe_off'] =True
config_cb['head_off'] =True
config_cb['body_off'] =True
config_cb['one_distribute_on'] =True
config_cb['drop_duplicate'] =True

In [ ]:
# 데이터 셋 만들기
data_cb = select_dataset(train,config_cb)
test_cb = select_dataset(test, config_cb)

# feature, target 분리
X = data_cb.drop(columns = ["label"])
y = data_cb['label']

In [ ]:
def fit_model(X,y,X_test,i,depth,bagging_temperature,learning_rate,subsample):
    cat_model = CatBoostClassifier(depth = int(depth),
                                  bagging_temperature=bagging_temperature,
                                  learning_rate=learning_rate,
                                  random_state=1339,#
                                  verbose=0,#
                                  subsample=subsample,
                                  eval_metric='F1'
                                  ).fit(X,y)
    cat_predict = cat_model.predict(X_test)
    return cat_predict

In [ ]:
def CAT_cv(depth,bagging_temperature,learning_rate,subsample):

    # KOLD CV
    kf=KFold(5,shuffle=True,random_state=1339)
    score = 0
    i=0
    for train_index,val_index in kf.split(X,y):
        i+=1
        X_train,X_val=X.iloc[train_index],X.iloc[val_index]
        y_train,y_val=y.iloc[train_index],y.iloc[val_index]
        cat_predict = fit_model(np.array(X_train),np.array(y_train),np.array(X_val),i,depth,bagging_temperature,learning_rate,subsample)
        score += metrics.f1_score(np.array(y_val),cat_predict)

    return score/5

- 베이지안 옵티마이저를 이용한 하이퍼 파라미터 써치

In [ ]:
# pbounds : 하이퍼 파라미터들의 범위 설정하기

pbounds = { 'depth': (1, 16),
            'bagging_temperature': (1, 10),
            'learning_rate': (0.01, 1.0),
            'subsample' : (0.01,1),
            }

In [ ]:
bo = BayesianOptimization(f = CAT_cv, pbounds = pbounds, random_state = 1,verbose = 2)
bo.maximize(init_points = 5, n_iter = 10,acq = 'ei',xi = 0.01)
#1시간 30분 결과

In [ ]:
# 앙상블 할 각각의 모형 정의하기
cat1 = CatBoostClassifier(bagging_temperature =5.197, depth = 11, learning_rate = 0.2278, subsample = 0.5169,random_state=1,verbose = 0,eval_metric='F1')
cat2 = CatBoostClassifier(bagging_temperature =4.186, depth = 11, learning_rate = 0.01, subsample = 0.7078,random_state=1,verbose = 0,eval_metric='F1')

# 앙상블 모형 정의하기
eclf = VotingClassifier(estimators=[
         ('cat1', cat1), ('cat2', cat2)],
         voting='soft')

# 모델 학습하기
eclf.fit(X,y)

# 예측한 후 저장하기
predict = eclf.predict(test_cb)


In [ ]:
X.columns

In [ ]:
test_cb.columns

In [ ]:
predict2 = np.where(predict==1,'benign','malicious')

In [ ]:
predict2

array(['benign', 'benign', 'benign', ..., 'benign', 'malicious',
       'malicious'], dtype='<U9')

In [ ]:
submission = pd.read_csv('sample_submission.csv')
submission['label'] = predict2
submission.to_csv("cat_esb_best2.csv", index = False)

In [ ]:
submission

,id,label
0,0,malicious
1,1,benign
2,2,benign
3,3,benign
4,4,benign
...,...,...
2436,2436,benign
2437,2437,malicious
2438,2438,malicious
2439,2439,benign


### 2) LGBM-C 모델 이용
- 파라미터가 100개 넘게 있어서 이것저것 쓰기 좋음
- 점수 1등의 경우의 수

In [ ]:
from sklearn.metrics import f1_score

from lightgbm import LGBMClassifier

- (False, True, True, False, False, False, False, True, False)	0.959752

In [ ]:
config_lgbmc = dict()
config_lgbmc['one_column_on'] =False 
config_lgbmc['url_len_off'] =True
config_lgbmc['url_entropy_off'] =True
config_lgbmc['url_port_off'] =False 
config_lgbmc['iframe_off'] =False
config_lgbmc['head_off'] =False
config_lgbmc['body_off'] =False
config_lgbmc['one_distribute_on'] =True
config_lgbmc['drop_duplicate'] =False

In [ ]:
# 데이터 셋 만들기
data_lgbmc = select_dataset(train,config_lgbmc)
test_lgbmc = select_dataset(test,config_lgbmc)

# feature, target 분리
X = data_lgbmc.drop(columns = ['label'])
y = data_lgbmc['label']

In [ ]:
def fit_model(X,y,X_test,i,max_depth, num_leaves, min_child_samples, min_child_weight, subsample, colsample_bytree, max_bin, reg_lambda, reg_alpha):
  params = {
      "n_estimators" : 500, "learning_rate": 0.02,
      'max_depth': int(round(max_depth)),
      'num_leaves': int(round(num_leaves)),
      'min_child_samples': int(round(min_child_samples)),
      'min_child_weight': int(round(min_child_weight)),
      'subsample': max(min(subsample, 1), 0), # 0 과 1 사이
      'colsample_bytree': max(min(colsample_bytree, 1), 0),
      'max_bin':  max(int(round(max_bin)),10),
      'reg_lambda': max(reg_lambda,0),
      'reg_alpha': max(reg_alpha, 0)
  }
  lgb_model = LGBMClassifier(**params).fit(X,y)
  lgb_predict = lgb_model.predict(X_test)
  return lgb_predict

In [ ]:
def lgb_cv(max_depth, num_leaves, min_child_samples, min_child_weight, subsample, colsample_bytree, max_bin, reg_lambda, reg_alpha):

    # KOLD CV
    kf=KFold(5,shuffle=True,random_state=1339)
    score = 0
    i=0
    for train_index,val_index in kf.split(X,y):
        i+=1
        X_train,X_val=X.iloc[train_index],X.iloc[val_index]
        y_train,y_val=y.iloc[train_index],y.iloc[val_index]
        lgb_predict = fit_model(np.array(X_train),np.array(y_train),np.array(X_val),i,max_depth,
                                num_leaves, min_child_samples, min_child_weight, subsample,
                                colsample_bytree, max_bin, reg_lambda, reg_alpha)
        score += metrics.f1_score(np.array(y_val),lgb_predict)

    return score/5

- 베이지안 옵티마이저를 이용한 하이퍼 파라미터 써치

In [ ]:
# pbounds : 하이퍼 파라미터들의 범위 설정하기

pbounds = { 'max_depth':(6, 16), #트리 최대 깊이
    'num_leaves':(24,64), #트리 하나에 최대 잎 개수
    'min_child_samples': (10, 200), #하나의 잎에 최소 데이터 개수 (오버피팅 대응)
    'min_child_weight': (1, 50), #하나의 잎에 최소 sum hessian
    'subsample': (0.5, 1), #
    'colsample_bytree': (0.5, 1), #피처의 50% 를 트레이닝 전에 선택
    'max_bin': (10, 500), #bins 의 최대 개수
    'reg_lambda': (0.001, 10), #L2 regularization
    'reg_alpha': (0.01, 50) #L1 regularization 
    }

In [ ]:
bo = BayesianOptimization(f = lgb_cv, pbounds = pbounds, random_state = 409)
bo.maximize(init_points = 5, n_iter = 50)

|   iter    |  target   | colsam... |  max_bin  | max_depth | min_ch... | min_ch... | num_le... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------
| 1         | 0.9673    | 0.6056    | 445.9     | 6.33      | 170.5     | 10.25     | 34.32     | 8.534     | 4.725     | 0.9298    |
| 2         | 0.9523    | 0.942     | 351.8     | 9.163     | 192.6     | 6.67      | 25.24     | 33.69     | 8.497     | 0.9114    |
| 3         | 0.9505    | 0.9263    | 159.6     | 6.488     | 44.3      | 14.36     | 60.0      | 38.83     | 8.709     | 0.6008    |
| 4         | 0.9535    | 0.9505    | 401.8     | 8.564     | 82.21     | 17.27     | 35.91     | 31.79     | 2.704     | 0.5523    |
| 5         | 0.9485    | 0.6794    | 74.99     | 9.591     | 146.8     | 49.07     | 52.41     | 29.43     | 1.073     | 0.8812    |
| 6         | 0.9636    | 0.513     | 418.8     | 14.0      | 

KeyboardInterrupt: ignored

In [ ]:
test_lgbmc.drop(columns=['id'], inplace=True)

In [ ]:
test_lgbmc.shape

(2441, 17)

In [ ]:
# 앙상블 할 각각의 모형 정의하기
 
cat1 = LGBMClassifier(colsample_bytree =1.0, max_bin = 433, max_depth = 16,
                      min_child_samples = 40,min_child_weight=1.0,num_leaves= 64,reg_alpha=0.01,
                      reg_lambda=0.001,subsample=1.0, random_state=409,verbose = 0,eval_metric='F1')
cat2 = LGBMClassifier(colsample_bytree =1.0, max_bin = 457, max_depth = 16,
                      min_child_samples = 57,min_child_weight=1.0,num_leaves=64,reg_alpha=0.01,
                      reg_lambda=0.001,subsample=0.5, random_state=409,verbose = 0,eval_metric='F1')
cat3 = LGBMClassifier(colsample_bytree =1.0, max_bin = 411, max_depth = 16,
                      min_child_samples = 117,min_child_weight=1.0,num_leaves=24,reg_alpha=0.01,
                      reg_lambda=0.001,subsample=0.5, random_state=409,verbose = 0,eval_metric='F1')
cat4 = LGBMClassifier(colsample_bytree =1.0, max_bin = 401, max_depth = 16,
                      min_child_samples = 132,min_child_weight=1.0,num_leaves=24,reg_alpha=0.01,
                      reg_lambda=0.001,subsample=1.0, random_state=409,verbose = 0,eval_metric='F1')
cat5 = LGBMClassifier(colsample_bytree =1.0, max_bin = 401, max_depth = 9,
                      min_child_samples = 170,min_child_weight=12,num_leaves=64,reg_alpha=0.01,
                      reg_lambda=0.001,subsample=1.0, random_state=409,verbose = 0,eval_metric='F1')

# 앙상블 모형 정의하기
eclf = VotingClassifier(estimators=[
         ('cat1', cat1), ('cat2', cat2), ('cat3', cat3), ('cat4', cat4), ('cat5', cat5)],
         voting='soft')

# 모델 학습하기
eclf.fit(X,y)

# 예측한 후 저장하기
pred2 = eclf.predict(test_lgbmc)

In [ ]:
pred2

array([1, 1, 1, ..., 0, 0, 1])

In [ ]:
pred = np.where(pred2==1,'benign','malicious')

In [ ]:
pred

array(['benign', 'benign', 'benign', ..., 'malicious', 'malicious',
       'benign'], dtype='<U9')

In [ ]:
submission = pd.read_csv('sample_submission.csv')
submission['label'] = pred
submission.to_csv("hmm(2).csv", index = False)

n_estimators= 1000, num_leaves=64, boost_from_average=True / False, eval_metric='F1', early_stopping_round